<a href="https://colab.research.google.com/github/MAN1108/github-test/blob/main/GPT-3.5-turbo-RAG-wwith_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
import openai
import os # Or the relevant LangChain modules
os.environ["OPENAI_API_KEY"] = "sk-proj-yH3vKkW04lp6lH4aCsoJT3BlbkFJFm3I8D2eDEOpL8uxg4cY"

In [4]:
!pip install python-dotenv
import dotenv
dotenv.load_dotenv()

False

In [5]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.3 MB/s eta 0:00:00


In [6]:
def load_pdf_files(directory_path):
    """Loads text from multiple PDFs in a directory."""
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.pdf'):
                file_path = os.path.join(root, file)
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
    return documents

directory_path = "/content/sample_data/harry"  # Replace with your directory
documents = load_pdf_files(directory_path)
print(len(documents))


4098


In [7]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [9]:
!pip install tiktoken
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(api_key="sk-proj-yH3vKkW04lp6lH4aCsoJT3BlbkFJFm3I8D2eDEOpL8uxg4cY"),
    by_text = False
)

embedded weaviate is already listening on port 8079


In [10]:
retriever = vectorstore.as_retriever()

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
only use the following pieces of retrieved context  if helpful to give one-shot answer to the question.
just give one-shot answer.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='You are an assistant for question-answering tasks.\nonly use the following pieces of retrieved context  if helpful to give one-shot answer to the question.\njust give one-shot answer.\nQuestion: {question}\nContext: {context}\nAnswer:\n'))]


In [41]:
import time
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "Why does Fudge refuse to believe Harry about Voldemortâ€™s return in 'Harry Potter and the Goblet of Fire'?"

#rag_chain.invoke(query)
start_time = time.time()
response =rag_chain.invoke(query)
#response = rag_chain({"query": query})
end_time = time.time()
total_time = int(end_time - start_time)
print(response)
print(f"Total calculation time: {total_time} seconds")



Fudge refuses to believe Harry about Voldemort's return because he is in denial and doesn't want to face the truth that would disrupt his comfortable world.
Total calculation time: 1 seconds


In [42]:
!pip install datasets
from datasets import Dataset

questions = ["What is the name of the magical plant that strangles anything that comes near it?",
             "What is the name of the spell used to create a protective shield around the caster?",
             "What is the name of the wand shop in Diagon Alley?",
             "Which Quidditch players fend off the Bludgers?",
             "What is the name of Filch's cat?",
             "What's Nearly Headless Nick's real name?",
             "What is the name of the spell used to immobilize a target?",
             "Which character is the head of the Malfoy family?",
             "What is the name of the ghost who haunts the Ravenclaw Tower?",
             "What was the name of the Potters' house-elf?",
             "Why does Horace Slughorn collect famous and talented wizards in 'Harry Potter and the Half-Blood Prince'?",
             "How does Sirius Blackâ€™s mentorship shape Harryâ€™s understanding of family and belonging?",
             "What is the significance of Harryâ€™s friendship with Luna Lovegood?",
             "Why does Moaning Myrtle haunt the girlsâ€™ bathroom in 'Harry Potter and the Chamber of Secrets'?",
             "Why does Fleur Delacour fall in love with Bill Weasley in 'Harry Potter and the Half-Blood Prince'?",
             "How does Dumbledoreâ€™s relationship with Harry influence his perception of power and responsibility?",
             "Why does Harry receive a Firebolt broomstick in 'Harry Potter and the Prisoner of Azkaban'?",
             "Why does Hagrid take care of magical creatures in 'Harry Potter and the Goblet of Fire'?",
             "Why does Harry face the Hungarian Horntail in 'Harry Potter and the Goblet of Fire'?",
             "Why does Fudge refuse to believe Harry about Voldemortâ€™s return in 'Harry Potter and the Goblet of Fire'?",

            ]
ground_truths = [["devil's snare"],
                 ["Protego"],
                 ["Ollivanders"],
                 ["The beaters"],
                 ["Mrs.Norris"],
                 ["Nicholas de Mimsy-Porpington"],
                 ["Immobilus"],
                 ["Lucius Malfoy"],
                 ["Gray Lady"],
                 ["Dobby"],
                 ["For his Slug Club, to network and be associated with successful individuals."],
                 ["It helps Harry find a sense of belonging and acceptance within the wizarding world, as he sees Sirius as a father figure and role model."],
                 ["It highlights the importance of accepting others for who they are and embracing diversity."],
                 ["Because she died there after looking into the eyes of the Basilisk."],
                 ["Because of his kindness, bravery, and support during the Triwizard Tournament."],
                 ["It teaches him that true power comes from compassion and empathy, and that with great power comes great responsibility."],
                 ["As a gift from Sirius Black and to replace his destroyed broom."],
                 ["It is his passion and he believes in teaching students about magical creatures."],
                 ["As part of the first task of the Triwizard Tournament."],
                 ["Because he fears the political implications and prefers to deny the truth."],
                ]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Embedded weaviate wasn't listening on ports http:8079 & grpc:50060, so starting embedded weaviate again
Started /root/.cache/weaviate-embedded: process ID 16761


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [43]:
dataset.to_csv("GPT_3.5-RAG_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

141278

In [46]:
import pandas as pd
test_df = pd.read_csv("/content/sample_data/GPT_3.5-RAG_dataset-Without_context.csv")

In [47]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

ground_truth = test_df['ground_truths'].tolist()
predictions = test_df['answer'].tolist()


def calculate_metrics(ground_truth, predictions):
  """
  Calculates and prints various evaluation metrics for question answering.

  Args:
      ground_truth: A list of strings containing the correct answers.
      predictions:  A list of strings containing the model's predicted answers.

  Returns:
      A dictionary containing the calculated metrics (accuracy, precision, recall, F1-score).
  """
  metrics = {}

  # Ensure equal list lengths (handle cases with missing predictions)
  ground_truth = ground_truth[:len(predictions)]
  predictions = predictions[:len(ground_truth)]
  predictions = [pred if pred is not None else "I'm not sure" for pred in predictions]
  # Calculate metrics
  metrics["accuracy"] = accuracy_score(ground_truth, predictions)
  metrics["precision"] = precision_score(ground_truth, predictions, average='weighted')  # Weighted precision for imbalanced datasets
  metrics["recall"] = recall_score(ground_truth, predictions, average='weighted')  # Weighted recall for imbalanced datasets
  metrics["f1"] = f1_score(ground_truth, predictions, average='weighted')  # Weighted F1-score for imbalanced datasets

  # Print Metric Results
  print("Evaluation Metrics:")
  for metric_name, value in metrics.items():
    print(f"{metric_name}: {value:.4f}")

  return metrics

# ... (Your code for generating ground_truth and predictions) ...

metrics = calculate_metrics(ground_truth, predictions)

Evaluation Metrics:
accuracy: 0.4000
precision: 0.4000
recall: 0.4000
f1: 0.4000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
!pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.2 MB/s eta 0:00:00


In [54]:
eval_df = pd.read_csv("/content/sample_data/GPT_3.5-RAG_dataset.csv")
eval_df1=eval_df
display(eval_df.head())

,question,contexts,answer,ground_truth
0,What is the name of the magical plant that str...,['THE BEETLE AT BAY \n\x91 547 \x91 “We hav...,devil's snare,devil's snare
1,What is the name of the spell used to create a...,['THE HALF-BLOOD PRINCE \n\x91 179 \x91 of S...,Shield Charm.,Protego
2,What is the name of the wand shop in Diagon Al...,['The Wandmaker\nfrom his pocket the two wands...,Ollivanders,Ollivanders
3,Which Quidditch players fend off the Bludgers?,['THE QUIDDITCH FINAL \n\x91 305 \x91 impair...,The beaters,The beaters
4,What is the name of Filch's cat?,"['The Thief\nof cats, as the same tent in whic...",Mrs. Norris,Mrs. Norris


In [55]:
from datasets import Dataset
def assemble_ragas_dataset(input_df):
    question_list, truth_list, context_list = [], [], []
    question_list = input_df.question.astype(str).to_list()
    truth_list = eval_df.ground_truth.astype(str).to_list()
    context_list = input_df.contexts.astype(str).to_list()
    context_list = [[contexts] for contexts in context_list]
    rag_answer_list = input_df.astype(str).answer.to_list()
    ragas_ds = Dataset.from_dict({"question": question_list,
                            "contexts": context_list,
                            "answer": rag_answer_list,
                            "ground_truth": truth_list
                            })
    return ragas_ds
ragas_input_ds = assemble_ragas_dataset(eval_df)
display(ragas_input_ds)

Dataset({
    features: ['question', 'contexts', 'answer', 'ground_truth'],
    num_rows: 20
})

In [56]:
import os, openai, pprint
from openai import OpenAI

# Save the api key in an env variable.
os.environ['OPENAI_API_KEY'] = 'sk-proj-yH3vKkW04lp6lH4aCsoJT3BlbkFJFm3I8D2eDEOpL8uxg4cY'

# Choose the metrics you want to see.
from ragas.metrics import ( context_recall, context_precision, faithfulness, answer_relevancy )
metrics = ['context_recall', 'context_precision', 'faithfulness', 'answer_relevancy']

# Change the llm-as-critic.
from ragas.llms import llm_factory
LLM_NAME = "gpt-3.5-turbo"
ragas_llm = llm_factory(model=LLM_NAME)

# Also change the embeddings.
from langchain_openai.embeddings import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
lc_embeddings = OpenAIEmbeddings( model="text-embedding-3-small", dimensions=512 )
ragas_emb = LangchainEmbeddingsWrapper(embeddings=lc_embeddings)

# Change the default models used for each metric.
for metric in metrics:
    globals()[metric].llm = ragas_llm
    globals()[metric].embeddings = ragas_emb

# Evaluate the dataset.
from ragas import evaluate
ragas_result = evaluate( ragas_input_ds,
    metrics=[ context_precision, context_recall, faithfulness, answer_relevancy],
    llm=ragas_llm,
)

Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

In [57]:
ragas_output_df = ragas_result.to_pandas()
ragas_output_df

,question,contexts,answer,ground_truth,context_precision,context_recall,faithfulness,answer_relevancy
0,What is the name of the magical plant that str...,[['THE BEETLE AT BAY \n\x91 547 \x91 “We ha...,devil's snare,devil's snare,1.0,1.00,1.000000,0.459877
1,What is the name of the spell used to create a...,[['THE HALF-BLOOD PRINCE \n\x91 179 \x91 of ...,Shield Charm.,Protego,1.0,0.00,1.000000,0.431784
2,What is the name of the wand shop in Diagon Al...,[['The Wandmaker\nfrom his pocket the two wand...,Ollivanders,Ollivanders,1.0,1.00,1.000000,0.589374
3,Which Quidditch players fend off the Bludgers?,[['THE QUIDDITCH FINAL \n\x91 305 \x91 impai...,The beaters,The beaters,1.0,0.00,1.000000,0.526244
4,What is the name of Filch's cat?,"[['The Thief\nof cats, as the same tent in whi...",Mrs. Norris,Mrs. Norris,1.0,1.00,1.000000,0.328932
5,What's Nearly Headless Nick's real name?,[['CHAPTER THIRTY-EIGHT \n\x91 860 \x91 “No —...,Nicholas de Mimsy-Porpington,Nicholas de Mimsy-Porpington,1.0,1.00,1.000000,0.447119
6,What is the name of the spell used to immobili...,[['CHAPTER THIRTY-ONE \n\x91 608 \x91 own for...,Impediment Curse.,Immobilus,1.0,0.00,1.000000,0.417337
7,Which character is the head of the Malfoy family?,"[['Chapter 1\n“My Lord?”\n“Your wand, Lucius. ...",Lucius Malfoy,Lucius Malfoy,1.0,1.00,0.000000,0.503668
8,What is the name of the ghost who haunts the R...,[['The Battle of Hogwarts\ndiadem that had elu...,Gray Lady,Gray Lady,1.0,1.00,1.000000,0.311642
9,What was the name of the Potters' house-elf?,[['CHAPTER TWENTY \n\x91 434 \x91 Harry could...,Dobby,Dobby,1.0,1.00,0.000000,0.475386
